# Reader

The **Reader pattern** is a way of stringing functions together when those functions are all **awaiting a shared input from a common environment**. In application development, programmers often need to pass around information that is required intermittently or universally, such as configuration data or a database connection. Instead of passing this information as an explicit argument to almost every function—which makes code harder to read and maintain—the Reader pattern allows this "read-only" environment to be shared implicitly.

### Core Intuition and Motivation
The core intuition behind Reader is that it provides a way to **abstract out function application**, allowing you to perform computation in terms of an argument that has not yet been supplied. It is particularly useful for:
*   **Avoiding "argument clutter":** It prevents the type of every function from being dominated by a universal environment argument.
*   **Constant values:** It is most often used when a constant value obtained from outside the program needs to be used by a wide variety of functions.

### Technical Foundation
The Reader pattern is built upon the fact that **partially applied functions**—specifically the type `(->) r`—already possess **Functor, Applicative, and Monad instances**. 

*   **Functor of functions:** When you `fmap` a function over another, it is identical to **function composition**. The structure being lifted over is the argument type `r`, and the value being transformed is the result of the function.
*   **Applicative of functions:** This context allows you to pass a **single shared argument to multiple functions in parallel** and then combine their results. This is often used when two functions share the same input and you want to apply a third function to their results to reach a final value.
*   **Monad of functions:** The Monad instance allows for **chaining functions** that all wait for the same environment. Using `do` syntax with functions allows you to bind variables to the eventual results of those functions once the shared input is provided.

### The Reader Type
While the pattern can be used with raw functions, Haskell provides a specific **newtype wrapper** for clarity:

```haskell
newtype Reader r a = Reader { runReader :: r -> a }
```

In this definition, **`r` represents the environment type** being "read," and **`a` is the result type** of the computation. The `runReader` accessor is used to extract the underlying function from the wrapper so it can be applied to an environment value.

### Practical Usage
In production Haskell code, you will more commonly encounter **`ReaderT`**, which is the **monad transformer** version of Reader. Reader rarely stands alone; it is typically part of a "stack" of monads (such as combining Reader, Either, and IO) to provide a robust environment that handles shared state, error catching, and input/output simultaneously.

## Monad Reader

The **Monad instance for Reader** is a way of chaining functions that are all **awaiting a single shared input from a common environment**. It allows you to perform computations in terms of an argument that has not yet been supplied, effectively abstracting out function application.

### The Technical Structure
In the context of the Reader pattern, the monadic structure `m` is the **partially applied function type `((->) r)`**, where `r` is the environment being "read". When we specialize the Monad methods for this type, they look like this:

*   **`return :: a -> r -> a`**: This is identical to the `const` function; it takes a value and creates a function that ignores the environment and returns that value.
*   **`(>>=) :: (r -> a) -> (a -> r -> b) -> (r -> b)`**: The bind operator takes a function awaiting an `r`, and a second function that produces a new Reader computation. It pipes the shared argument `r` through the chain.

The implementation of bind for functions can be summarized as:
`f >>= k = \r -> k (f r) r`.

### Examples of the Reader Monad

#### 1. Basic `do` Syntax with Arithmetic
Consider two functions, `boop = (*2)` and `doop = (+10)`. We can combine them using the Reader Monad so they both receive the same input:

```haskell
boopDoop :: Integer -> Integer
boopDoop = do
  a <- boop    -- a is the result of (input * 2)
  b <- doop    -- b is the result of (input + 10)
  return (a + b)
```
In this example, as soon as an input is provided to `boopDoop`, it fills the empty slots in both `boop` and `doop`. The results are bound to `a` and `b`, then added together.

#### 2. Building Records from a Shared Environment
The Reader Monad is often used to extract multiple pieces of information from a single complex record. Imagine we have a `Person` and we want to create a `Dog` using the person's data:

```haskell
getDogRM :: Person -> Dog
getDogRM = do
  name <- dogName    -- dogName is a function: Person -> DogName
  addy <- address    -- address is a function: Person -> Address
  return $ Dog name addy
```
Here, `dogName` and `address` are functions "reading" from the same `Person` environment. The Monad instance handles passing the `Person` argument to both functions behind the scenes.

#### 3. Abstracting Function Application
A more abstract example shows how `bind` works under the hood to combine two functions, `foo` and `bar`, that are waiting for the same argument `r`:

```haskell
frooty' :: Num a => [a] -> ([a], Int)
frooty' = \r -> bar (foo r) r
```
This pattern is exactly what `bind` (or `fooBind` in the sources) generalizes: it takes an environment `r`, applies the first function to it, and then passes both the result and the original `r` to the next function.

### Summary of Utility
The Reader Monad is most useful when you have a **constant value** (like a configuration or database connection) obtained from outside the program that needs to be an argument to a large number of functions. Using the Monad instance allows you to **avoid passing that argument explicitly** to every single function call.

## Short Exercise: Warming Up


We’ll be doing something here very similar to what you saw above, to give you practice and try to develop a feel or intuition for what is
to come. These are similar enough to what you just saw that you can almost copy and paste, so try not to overthink them too much. First, start a file oﬀ like this:

```haskell
import Data.Char


cap :: [Char] -> [Char]
cap xs = map toUpper xs

rev :: [Char] -> [Char]
rev xs = reverse xs
```

Two simple functions with the same type, taking the same type of input. We could compose them, using `(.)` or `fmap`:

```haskell
composed :: [Char] -> [Char]
composed = undefined

fmapped :: [Char] -> [Char]
fmapped = undefined
```

The output of those two should be identical: one string that is made all uppercase and reversed, like this:

```
Prelude> composed "Julie"
"EILUJ"
Prelude> fmapped "Chris"
"SIRHC"
```

In [1]:
import Data.Char


cap :: [Char] -> [Char]
cap xs = map toUpper xs

rev :: [Char] -> [Char]
rev xs = reverse xs

composed :: [Char] -> [Char]
composed = rev . cap

fmapped :: [Char] -> [Char]
fmapped = fmap rev cap

Line 5: Eta reduce
Found:
cap xs = map toUpper xs
Why not:
cap = map toUpperLine 8: Eta reduce
Found:
rev xs = reverse xs
Why not:
rev = reverse

In [2]:
composed "Julie"

"EILUJ"

In [3]:
fmapped "Chris"

"SIRHC"

Now we want to return the results of cap and rev both, as a tuple, like this:

```
Prelude> tupled "Julie"
("JULIE","eiluJ")
-- or

Prelude> tupled' "Julie"
("eiluJ","JULIE")
```

We will want to use an Applicative here. The type will look like this:
```haskell
tupled :: [Char] -> ([Char], [Char])
```

There is no special reason such a function needs to be monadic,
but let’s do that, too, to get some practice. Do it one time using do
syntax; then try writing a new version using `(>>=)`. The types will be
the same as the type for tupled.

In [12]:
tupled :: [Char] -> ([Char], [Char])
tupled str = (cap str, rev str)

In [14]:
tupled "Julie"

("JULIE","eiluJ")

In [21]:
tupledDo :: [Char] -> ([Char], [Char])
tupledDo = do
    c <- cap
    r <- rev
    return (c, r)

In [22]:
tupledDo "Julie"

("JULIE","eiluJ")

In [32]:
tupledMonadic :: [Char] -> ([Char], [Char])
tupledMonadic = cap >>= (rev >>= return (,))